In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

In [2]:
# add test data for classification
data = pd.read_csv('./data/ciga_classification.csv')
data.head()

,text,label
0,烟叶的主要成分包括尼古丁、糖和蛋白质。,relevant
1,世界卫生组织建议尽量减少吸烟以降低健康风险。,irrelevant
2,烟草的发酵过程对于卷烟的最终风味至关重要。,relevant
3,每天坚持跑步有助于增强体质。,irrelevant
4,卷烟的滤嘴设计可以影响吸烟时的口感和吸入的有害物质量。,relevant


In [3]:
df = data

In [4]:
import spacy

# 加载预训练的spaCy中文模型
nlp = spacy.load("zh_core_web_sm")

def preprocess_text(text):
    # 使用spaCy进行分词
    doc = nlp(text)
    # 去除连接词和标点符号
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    # 将tokens连接成一个字符串
    return " ".join(tokens)

# 假设df已经存在并且有一列名为'text'
df['processed_text'] = df['text'].apply(preprocess_text)
df

,text,label,processed_text
0,烟叶的主要成分包括尼古丁、糖和蛋白质。,relevant,烟叶 成分 包括 尼古丁 糖 蛋白质
1,世界卫生组织建议尽量减少吸烟以降低健康风险。,irrelevant,世界 卫生 组织 建议 减少 吸烟 降低 健康 风险
2,烟草的发酵过程对于卷烟的最终风味至关重要。,relevant,烟草 发酵 过程 卷烟 最终 风味 至关重要
3,每天坚持跑步有助于增强体质。,irrelevant,跑步 有助于 增强 体质
4,卷烟的滤嘴设计可以影响吸烟时的口感和吸入的有害物质量。,relevant,卷烟 滤嘴 设计 影响 吸烟 时 口感 吸入 有害物 质量
5,制造卷烟时需要严格控制温度和湿度以确保质量。,relevant,制造 卷烟 时 控制 温度 湿度 确保 质量
6,吸烟可能会导致多种疾病，包括肺癌和心脏病。,irrelevant,吸烟 导致 多种 疾病 包括 肺癌 心脏病
7,使用不同种类的烟叶可以调配出不同风味的卷烟产品。,relevant,种类 烟叶 调配 风味 卷烟 产品
8,烟草种植过程中需要注意病虫害防治以保证烟叶质量。,relevant,烟草 种植 过程 中 病虫害 防治 保证 烟叶 质量
9,多喝水有助于保持身体健康。,irrelevant,喝 水 有助于 身体 健康


In [5]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df['processed_text'], df['label'], test_size=0.3, random_state=42)

# Create a pipeline with TF-IDF and Naive Bayes classifier
pipeline = make_pipeline(
    TfidfVectorizer(),
    MultinomialNB()
)

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

  irrelevant       1.00      0.50      0.67         2
    relevant       0.80      1.00      0.89         4

    accuracy                           0.83         6
   macro avg       0.90      0.75      0.78         6
weighted avg       0.87      0.83      0.81         6



In [6]:
# Function to classify new text

def classify_text(text):
    processed_text = preprocess_text(text)
    prediction = pipeline.predict([processed_text])
    return prediction[0]

In [7]:
new_texts = [
    "烟叶需要分类和处理。",
    "检查库存的库存水平。",
    "新型香烟过滤技术提高了烟雾质量。",
    "吸烟有害健康",
    "烟草花叶病毒是烟草的一种病毒病。",
    "电子烟的使用越来越受到关注。",
    "产品的包装标准对市场竞争力有影响。",
    "烟草属于危险品，需要妥善保管。"
    "烤烟是一种烟草品种。",
    "烟草生产需要严格遵守环保法规。",
    "香烟的成分包括烟草和添加剂。",
    "烟草的种植需要适宜的土壤和气候条件。",
    "烟草的加工过程需要控制温湿度。",
    "香烟的生产需要专业的生产设备。",
    "烟叶需要分类和处理。",
    "检查库存的库存水平。",
    "新型香烟过滤技术提高了烟雾质量。",
    "吸烟有害健康",
    "烟草花叶病毒是烟草的一种病毒病。",
    "电子烟的使用越来越受到关注。",
    "产品的包装标准对市场竞争力有影响。",
    "烟草属于危险品，需要妥善保管。"
]

relevant_texts = []

for text in new_texts:
    print(f"Text: {text}\nClassification: {classify_text(text)}\n")
    if classify_text(text) == 'relevant':
        relevant_texts.append(text)

print(f"Relevant Texts: {relevant_texts}")

Text: 烟叶需要分类和处理。
Classification: relevant

Text: 检查库存的库存水平。
Classification: relevant

Text: 新型香烟过滤技术提高了烟雾质量。
Classification: relevant

Text: 吸烟有害健康
Classification: irrelevant

Text: 烟草花叶病毒是烟草的一种病毒病。
Classification: relevant

Text: 电子烟的使用越来越受到关注。
Classification: relevant

Text: 产品的包装标准对市场竞争力有影响。
Classification: relevant

Text: 烟草属于危险品，需要妥善保管。烤烟是一种烟草品种。
Classification: relevant

Text: 烟草生产需要严格遵守环保法规。
Classification: relevant

Text: 香烟的成分包括烟草和添加剂。
Classification: relevant

Text: 烟草的种植需要适宜的土壤和气候条件。
Classification: relevant

Text: 烟草的加工过程需要控制温湿度。
Classification: relevant

Text: 香烟的生产需要专业的生产设备。
Classification: relevant

Text: 烟叶需要分类和处理。
Classification: relevant

Text: 检查库存的库存水平。
Classification: relevant

Text: 新型香烟过滤技术提高了烟雾质量。
Classification: relevant

Text: 吸烟有害健康
Classification: irrelevant

Text: 烟草花叶病毒是烟草的一种病毒病。
Classification: relevant

Text: 电子烟的使用越来越受到关注。
Classification: relevant

Text: 产品的包装标准对市场竞争力有影响。
Classification: relevant

Text: 烟草属于危险品，需要妥善保管。
Classification: relevant

Relevant

In [8]:
import spacy

# Load the transformer-based Chinese model
nlp = spacy.load("zh_core_web_sm")

def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities


for text in relevant_texts:
    entities = extract_entities(text)
    print(f"Text: {text}\nEntities: {entities}\n")

Text: 烟叶需要分类和处理。
Entities: []

Text: 检查库存的库存水平。
Entities: []

Text: 新型香烟过滤技术提高了烟雾质量。
Entities: []

Text: 烟草花叶病毒是烟草的一种病毒病。
Entities: []

Text: 电子烟的使用越来越受到关注。
Entities: []

Text: 产品的包装标准对市场竞争力有影响。
Entities: []

Text: 烟草属于危险品，需要妥善保管。烤烟是一种烟草品种。
Entities: []

Text: 烟草生产需要严格遵守环保法规。
Entities: []

Text: 香烟的成分包括烟草和添加剂。
Entities: []

Text: 烟草的种植需要适宜的土壤和气候条件。
Entities: []

Text: 烟草的加工过程需要控制温湿度。
Entities: [('温湿度', 'GPE')]

Text: 香烟的生产需要专业的生产设备。
Entities: []

Text: 烟叶需要分类和处理。
Entities: []

Text: 检查库存的库存水平。
Entities: []

Text: 新型香烟过滤技术提高了烟雾质量。
Entities: []

Text: 烟草花叶病毒是烟草的一种病毒病。
Entities: []

Text: 电子烟的使用越来越受到关注。
Entities: []

Text: 产品的包装标准对市场竞争力有影响。
Entities: []

Text: 烟草属于危险品，需要妥善保管。
Entities: []

